# Moosic data & database management


* requirements file is `requirements_dev`


In [1]:


# IMPORT LIBRARIES


try:

    import os
    import numpy as np
    import pandas as pd
    import random 
    import time

    # databases - sql
    import psycopg2
    import sqlalchemy
    from dotenv import dotenv_values, load_dotenv
    from sqlalchemy import create_engine

    # visualisation
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import ListedColormap

    # pipeline
    from sklearn.pipeline import Pipeline
    from sklearn.pipeline import make_pipeline

    # custom imports
    from sql_functions import get_sql_config




except ImportError as error:
    print(f"Installation of the required dependencies necessary! {error}")

    %pip install numpy
    %pip install pandas
    %pip install psycopg2-binary
    %pip install python-dotenv
    %pip install sqlalchemy
    %pip install seaborn
    %pip install matplotlib


    print(f"Successful installation of the required dependencies necessary")


import warnings
warnings.filterwarnings('ignore')





## Creating a connection to a PostgreSQL database with Python


In [2]:
# Read database string from .env file 

#load_dotenv()
#db_url = os.getenv('db_url')


In [3]:
# function definition for get_connection_engine and get_dataframe() using read_sql_query()
# - call the function imported get_sql_config() and save the results to a variable
# - function to get connection engine to database
# - get data from database using the engine and an sql query


def get_conn_engine():

    # get database url, db_url

    sql_config_data = get_sql_config()

    db_url = f'''postgresql://{sql_config_data['user']}:{sql_config_data['password']}@{sql_config_data['host']}:{sql_config_data['port']}/{sql_config_data['database']}'''

    # create a connection engine to the PostgreSQL server

    try:
        engine = create_engine(db_url)
    except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
        print(error)

    return engine


def get_dataframe(sql_query, conn_engine):
    ''' 
    Connect to the PostgreSQL database server, 
    run query and return data as a pandas dataframe
    '''
    moosic_data = pd.read_sql_query(sql=sql_query, con=conn_engine)

    return moosic_data

# Retrieving data from database




In [4]:
# global variables like
# - database schema = 'public'
# - database connection engine = get_conn_engine()

schema = 'public'

conn_engine = get_conn_engine()




In [5]:
# queries : joins artist + tracks

sql_query_0 = f'''SELECT 
                        t.id AS track_id,
                        t.name AS track_name,
                        t.popularity AS track_popularity,
                        t.duration_ms AS track_duration_ms,
                        t.explicit AS track_explicit,
                        t.artists AS track_artists_name,
                        t.id_artists AS track_id_artists,
                        t.release_date AS track_release_date,
                        t.danceability AS track_danceability,
                        t.energy AS track_energy,
                        t.key AS track_key,
                        t.loudness AS track_loudness,
                        t.mode AS track_mode,
                        t.speechiness AS track_speechiness,
                        t.acousticness AS track_acousticness,
                        t.instrumentalness AS track_instrumentalness,
                        t.liveness AS track_liveness,
                        t.valence AS track_valence,
                        t.tempo AS track_tempo,
                        t.time_signature AS track_time_signature,
                        a.id AS artist_id,
                        a.name AS artist_name,
                        a.genres AS artist_genres,
                        a.followers AS artist_followers,
                        a.popularity AS artist_popularity

                    FROM
                        {schema}.track_spotify_data AS t
                    LEFT JOIN
                        {schema}.artist_spotify_data AS a
                    ON
                        t.id_artists = a.id;''' 

spotify_data_left =  get_dataframe(sql_query_0, conn_engine)


sql_query_1 = f'''SELECT * 
                FROM
                    {schema}.spotify_data'''


spotify_data = get_dataframe(sql_query_1, conn_engine)





In [6]:
# display

display(spotify_data_left.head(2))

display(spotify_data.head(2))



,track_id,track_name,track_popularity,track_duration_ms,track_explicit,track_artists_name,track_id_artists,track_release_date,track_danceability,track_energy,...,track_instrumentalness,track_liveness,track_valence,track_tempo,track_time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity
0,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,...,0.0218,0.212,0.457,130.418,5,None,None,None,None,None
1,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,...,0.9180,0.104,0.397,169.980,3,None,None,None,None,None


,track_id,track_name,track_popularity,track_duration_ms,track_explicit,track_artists_name,track_id_artists,track_release_date,track_danceability,track_energy,...,track_instrumentalness,track_liveness,track_valence,track_tempo,track_time_signature,artist_id,artist_name,artist_genres,artist_followers,artist_popularity
0,56NEvRJZMxL5d0ZkIXzV4D,Summerhead - Remastered 2006,20,217213,0,['Cocteau Twins'],['5Wabl1lPdNOeIn0SQ5A1mp'],2018-10-05,0.437,0.700,...,0.000021,0.242,0.5570,95.002,4,None,None,None,None,None
1,721MGE6dhsVO9DOlTppBCt,Pur - Remastered 2006,20,300893,0,['Cocteau Twins'],['5Wabl1lPdNOeIn0SQ5A1mp'],2018-10-05,0.290,0.592,...,0.000205,0.110,0.0715,155.760,4,None,None,None,None,None


In [7]:
# queries

# count of tracks in initial combined spotify data : artists + tracks
sql_query_2 = f'''SELECT count(*) 
                    FROM {schema}.spotify_data;'''

sql_query_2_df  = get_dataframe(sql_query_2, conn_engine)

display(sql_query_2_df.shape)
sql_query_2_df.head(2)



(1, 1)

,count
0,586672


In [8]:

# count of tracks in processed moosic data
sql_query_3 = f'''SELECT count(*) 
                    FROM {schema}.moodsic_data;'''



sql_query_3_df  = get_dataframe(sql_query_3, conn_engine)

display(sql_query_3_df.shape)
sql_query_3_df.head(2)



(1, 1)

,count
0,95032


In [9]:

# query tracks based on release date
sql_query_4 = f'''SELECT
                    track_name,
                    track_release_date
                FROM
                    {schema}.spotify_data
                WHERE
                    track_release_date BETWEEN '2004-01-01' AND '2012-06-30';'''

sql_query_4_df  = get_dataframe(sql_query_4, conn_engine)

display(sql_query_4_df.shape)
sql_query_4_df.head(2)



(78029, 2)

,track_name,track_release_date
0,You Only Live Once,2006-01-03
1,What Tomorrow Knows/Garden of Grey - live at t...,2008-07-13


In [10]:
# query specific columns


sql_query_5 = f'''SELECT track_id, artist_name,
                    energy, valence,
                    core_genres , mood_goal 
                FROM {schema}.moodsic_data;'''


sql_query_5_df  = get_dataframe(sql_query_5, conn_engine)

display(sql_query_5_df.shape)
sql_query_5_df.head(2)



(95032, 6)

,track_id,artist_name,energy,valence,core_genres,mood_goal
0,2Q85gpya4yOCEcX0uKDzAs,Þursaflokkurinn,0.282,0.147,rock,depressed
1,38DwzePhZtPxZJYgGuY7IY,King Crimson,0.286,0.147,jazz,depressed


In [11]:
# groupby specific columns

sql_query_6 = f'''SELECT track_id, artist_name, 
                        energy, valence, 
                        core_genres, mood_goal
                FROM {schema}.moodsic_data
                GROUP BY track_id, artist_name, energy, valence, core_genres, mood_goal;'''

sql_query_6_df  = get_dataframe(sql_query_6, conn_engine)

display(sql_query_6_df.shape)
sql_query_6_df.head(2)



(95032, 6)

,track_id,artist_name,energy,valence,core_genres,mood_goal
0,000Npgk5e2SgwGaIsN3ztv,Talat Mahmood,0.145,0.494,pop,sad
1,000ZxLGm7jDlWCHtcXSeBe,"Meade ""Lux"" Lewis",0.483,0.854,blues,relaxed


In [12]:
# count of tracks based on mood_goal, relaxed 


sql_query_7 = f'''SELECT
                    count(*)
                FROM
                    {schema}.moodsic_data
                WHERE
                    mood_goal = 'relaxed';'''

sql_query_7_df  = get_dataframe(sql_query_7, conn_engine)

display(sql_query_7_df.shape)
sql_query_7_df.head(2)



(1, 1)

,count
0,11879


In [13]:


# average valence-energy for music tracks for each mood category 

sql_query_8 = f'''SELECT
                    mood_goal,
                    AVG(energy) AS avg_energy,
                    AVG(valence) AS avg_valence
                FROM
                    {schema}.moodsic_data
                GROUP BY
                    mood_goal
                ORDER BY
                    mood_goal;'''

sql_query_8_df  = get_dataframe(sql_query_8, conn_engine)

display(sql_query_8_df.shape)
sql_query_8_df




(8, 3)

,mood_goal,avg_energy,avg_valence
0,angry,0.613001,0.332649
1,calm,0.178278,0.650368
2,depressed,0.375973,0.302044
3,euphoric,0.857913,0.748308
4,happy,0.631043,0.749955
5,relaxed,0.396613,0.705392
6,sad,0.153407,0.259142
7,tense,0.879695,0.333283


In [14]:


# average valence-energy for music tracks in each genre for each mood category
sql_query_9 = f'''SELECT
                    mood_goal,
                    core_genres,
                    AVG(energy) AS avg_energy,
                    AVG(valence) AS avg_valence
                FROM
                    {schema}.moodsic_data
                GROUP BY
                    mood_goal,
                    core_genres
                ORDER BY
                    mood_goal,
                    core_genres;'''

sql_query_9_df  = get_dataframe(sql_query_9, conn_engine)

display(sql_query_9_df.shape)
sql_query_9_df.head(2)





(195, 4)

,mood_goal,core_genres,avg_energy,avg_valence
0,angry,alternative,0.622733,0.331587
1,angry,blues,0.608293,0.353427


In [15]:
# processed data and queries on all music data with mood target labels


sql_query_10 = f'''SELECT  track_id, artist_name,
							energy, valence,
							core_genres , mood_goal 
				FROM {schema}.moosic_data_processed;''' 

sql_query_11 = f'''SELECT count(*)
					FROM {schema}.moosic_data_processed'''



In [16]:
# display all music processed data


sql_query_10_df  = get_dataframe(sql_query_10, conn_engine)

display(sql_query_10_df.shape)
sql_query_10_df.head(2)




(536452, 6)

,track_id,artist_name,energy,valence,core_genres,mood_goal
0,2wAfHM7Whz67VFbdanhZlk,Bessie Smith,0.0423,0.211,blues,sad
1,2AZgaYZSwUosJD71J2N2Zo,Bessie Smith,0.0443,0.137,blues,sad


In [17]:
#  display count of all music processed data


sql_query_11_df  = get_dataframe(sql_query_11, conn_engine)

sql_query_11_df


,count
0,536452


In [18]:

moosic_sql_all = f'''SELECT *
					FROM {schema}.moosic_data_processed'''

moosic_data_all = get_dataframe(moosic_sql_all, conn_engine)

# get shape of the moosic dataframe

print(f"Moosic data (all): There are {moosic_data_all.shape[0]} observations and {moosic_data_all.shape[1]} feature variables ")
print('----------'*10)

moosic_data_all.head(2)




Moosic data (all): There are 536452 observations and 26 feature variables 
----------------------------------------------------------------------------------------------------


,artists_id,track_id,artist_name,track_name,genres,release_date,explicit,duration_ms,danceability,energy,...,liveness,valence,tempo,time_signature,followers,artist_popularity,track_popularity,main_genres,core_genres,mood_goal
0,5ESobCkc6JI4tIMxQttqeg,2wAfHM7Whz67VFbdanhZlk,Bessie Smith,Nobody Knows You When Youre Down and Out,"blues, harlem renaissance, jazz blues, traditi...",1923,0,177133,0.614,0.0423,...,0.183,0.211,89.822,4,116265,41,44,['blues'],blues,sad
1,5ESobCkc6JI4tIMxQttqeg,2AZgaYZSwUosJD71J2N2Zo,Bessie Smith,Taint Nobodys Bizness If I Do,"blues, harlem renaissance, jazz blues, traditi...",1923,0,206600,0.537,0.0443,...,0.152,0.137,80.468,4,116265,30,44,['blues'],blues,sad


In [19]:
## saving processed moosic modelling data - all 500k samples

moosic_data_all.to_csv('../data/processed/moosic_data_processed.csv', chunksize=len(moosic_data_all)//5, index=False)



In [20]:
# query data file for moosic - this is the prepocessed and balanced moosic data with respect to the moods


moosic_sql_query = f'''SELECT *
                    FROM {schema}.moodsic_data;'''


moosic_data = get_dataframe(moosic_sql_query, conn_engine)

# get shape of the moosic dataframe

print(f"Moosic data: There are {moosic_data.shape[0]} observations and {moosic_data.shape[1]} feature variables ")
print('----------'*10)

moosic_data.head(2)

Moosic data: There are 95032 observations and 52 feature variables 
----------------------------------------------------------------------------------------------------


,artists_id,track_id,artist_name,track_name,genres,release_date,explicit,duration_ms,danceability,energy,...,metal,other,pop,punk rock,r&b,reggae,rock,rockabilly,soul,techno
0,3GQZwJTi7jL6e2dLqvMXj5,2Q85gpya4yOCEcX0uKDzAs,Þursaflokkurinn,"Hættu að gráta, hringaná","classic icelandic pop, icelandic rock",1978-01-01,0,164397,0.540,0.282,...,0,0,0,0,0,0,1,0,0,0
1,7M1FPw29m5FbicYzS2xdpi,38DwzePhZtPxZJYgGuY7IY,King Crimson,Matte Kudasai,"album rock, art rock, classic rock, instrument...",1981-01-01,0,229320,0.181,0.286,...,0,0,0,0,0,0,0,0,0,0


In [21]:
## saving final moosic modelling data

moosic_data.to_csv('../data/processed/moosic_modelling_data.csv', chunksize=len(moosic_data)//5, index=False)

